In [2]:
import pybamm as pb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [4]:
ChemistryChen=pb.parameter_sets.Chen2020  # Ren2018  Chen2020_plating
Para_0=pb.ParameterValues(chemistry=ChemistryChen);
Model_0  = pb.lithium_ion.DFN() 

In [5]:
# parameter - dimensional 
D_e     = 3e-10;    # Li+ diffusivity in m2/s
D_EC    = 5e-10;    # EC  diffusivity in m2/s
D_cross = -1.5e-9;  # Cross diffusivity in m2/s

t_0plus = 0.3;      # Transference number for Li+
trans_i = -1.4;     # Transference number for EC  

F = 96485.33;       # Faraday constant in C/mol

c_init = 1000;      # Initial Li-ion concentration in electrolyte
c_EC_init= 4000;    # Initial EC  concentration in electrolyte 
c_tot_init = 6000;  # Initial total concentration in electrolyte

V_bar_SEI = 9.585e-5;  # SEI partial molar volume in m3/mol
V_bar_EC  = 6.667e-5;  # EC  partial molar volume in m3/mol
V_bar_Li  = 1.300e-5;  # Li+ partial molar volume in m3/mol

b = 1.5;    # Bruggman cofficient

R_n       = 5.86e-6   # Mean Neg particle radius in m
R_p       = 5.22e-6   # Mean Pos particle radius in m

# Should be a variable but now set as constant to simplify
epislon_n = 0.25;     # Neg porosity
epislon_s = 0.47;     # Sep porosity
epislon_p = 0.335;     # Sep porosity

epislon_n = pb.FullBroadcast(
                pb.Scalar(0.25), "negative electrode", )
epislon_s = pb.FullBroadcast(
        pb.Scalar(0.47), "separator", )
epislon_p = pb.FullBroadcast(
        pb.Scalar(0.335), "positive electrode", )
epislon = pb.concatenation(epislon_n, epislon_s, epislon_p )

# During Discharge: j_int_neg is positive; j_SEI is also positive; 
# but j_int_pos is negative
# Total Neg interfacial current density in A/m2
j_tot_n = pb.FullBroadcast(
                pb.Scalar(2.1), "negative electrode", )
j_tot_s = pb.FullBroadcast(
        pb.Scalar(0), "separator", )
# Total Pos interfacial current density in A/m2
j_tot_p = pb.FullBroadcast(
        pb.Scalar(-1.7), "positive electrode", )
j_tot = pb.concatenation(j_tot_n, j_tot_s, j_tot_p )

j_SEI_n = pb.FullBroadcast(
        pb.Scalar(1e-3), "negative electrode", )
j_SEI_s = pb.FullBroadcast(
        pb.Scalar(0), "separator", )
j_SEI_p = pb.FullBroadcast(
        pb.Scalar(0), "positive electrode", )
j_SEI   = pb.concatenation(j_SEI_n,j_SEI_s,j_SEI_p);      # SEI       interfacial current density in A/m2
# based on wip\Rio_Code\P3_R0\Check_eley_current_dens.ipynb
i_eley = 50;         # Electrolyte current density [A.m-2]

a_n = 3* epislon_n / R_n    # Neg Specific surface area
a_p = 3* epislon_p / R_p    # Neg Specific surface area
zero_s = pb.FullBroadcast(0, "separator")

EC_ratio_Rio =  Para_0.evaluate(Model_0.param.EC_ratio_Rio)  
gamma_e_EC_Rio = Para_0.evaluate(Model_0.param.gamma_e_EC_Rio)  
tau_discharge = Para_0.evaluate(Model_0.param.tau_discharge)  
tau_cross_Rio = Para_0.evaluate(Model_0.param.tau_cross_Rio)  
D_EC_Li_cross = Para_0.evaluate(Model_0.param.D_EC_Li_cross)  
tau_EC_Rio    = Para_0.evaluate(Model_0.param.tau_EC_Rio)  
D_EC          = Para_0.evaluate(Model_0.param.D_EC)  
Xi            = Para_0.evaluate(Model_0.param.Xi)  
Vmolar_Li     = Para_0.evaluate(Model_0.param.Vmolar_Li)  
c_ec_0_dim    = Para_0.evaluate(Model_0.param.c_ec_0_dim)  
gamma_e       = Para_0.evaluate(Model_0.param.gamma_e)   
Vmolar_EC     = Para_0.evaluate(Model_0.param.Vmolar_EC) 
Vmolar_CH2OCO2Li2= Para_0.evaluate(Model_0.param.Vmolar_CH2OCO2Li2) 
C_e= Para_0.evaluate(Model_0.param.C_e) 
e_ratio_Rio = Para_0.evaluate(Model_0.param.e_ratio_Rio) 
c_e_init_dimensional = Para_0.evaluate(Model_0.param.c_e_init_dimensional) 

AttributeError: 'LithiumIonParameters' object has no attribute 'EC_ratio_Rio'

In [ ]:
sign_2_n = pb.FullBroadcast(
        pb.Scalar(1), "negative electrode", )
sign_2_s = pb.FullBroadcast(
        pb.Scalar(0), "separator", )
sign_2_p = pb.FullBroadcast(
        pb.Scalar(0), "positive electrode", )
sign_2 = pb.concatenation(sign_2_n, sign_2_s, sign_2_p )
a = pb.concatenation(a_n, zero_s, a_p)

In [ ]:
model = pb.BaseModel()
c_e_n  = pb.Variable("Li+ concentration in negative electrode", domain="negative electrode")
c_EC_n = pb.Variable("EC concentration in negative electrode", domain="negative electrode")

c_e_s  = pb.Variable("Li+ concentration in separator", domain="separator")
c_EC_s = pb.Variable("EC concentration in separator", domain="separator")

c_e_p  = pb.Variable("Li+ concentration in positive electrode", domain="positive electrode")
c_EC_p = pb.Variable("EC concentration in positive electrode", domain="positive electrode")

c_EC = pb.concatenation(c_EC_n, c_EC_s, c_EC_p)
c_e  = pb.concatenation(c_e_n, c_e_s, c_e_p)
model.variables = {
    "Li+ concentration in negative electrode":c_e_n,
    "Li+ concentration in separator":c_e_s,
    "Li+ concentration in positive electrode":c_e_p,
    "EC concentration in negative electrode":c_EC_n,
    "EC concentration in separator":c_EC_s,
    "EC concentration in positive electrode":c_EC_p,
}

In [ ]:
model.variables.update(
    {
        "Li+ concentration": c_e, 
        "EC concentration": c_EC,
    }
)

In [ ]:
print(model.variables.keys())

dict_keys(['Li+ concentration in negative electrode', 'Li+ concentration in separator', 'Li+ concentration in positive electrode', 'EC concentration in negative electrode', 'EC concentration in separator', 'EC concentration in positive electrode', 'Li+ concentration', 'EC concentration'])


In [ ]:
model.length_scales = {
    "negative electrode": 8.52e-5,
    "separator": 1.2e-5,
    "positive electrode": 7.56e-5,
}
model.timescale = pb.Scalar(8.52e-5 ** 2 /  D_e) 

In [ ]:
model.rhs = {
    c_e: 
    -pb.div(  pb.grad(c_e)) / C_e 
    + e_ratio_Rio * gamma_e_EC_Rio * tau_discharge 
    / tau_cross_Rio * pb.div( D_EC_Li_cross * pb.grad(c_EC))
    + ( 1-0.4 )  * 1 
    -  sign_2 * (    c_e_init_dimensional * Vmolar_Li   * 1         )
    -  (    
        c_e_init_dimensional / gamma_e * (
        Vmolar_EC-0.5*Vmolar_CH2OCO2Li2) * a * 1)  ,

    c_EC:
    epislon**b *D_cross*(c_EC_init/c_tot_init)*pb.div(pb.grad(c_e))
    + epislon**b *D_EC*pb.div(pb.grad(c_EC))
    + sign_2 * (
        a*trans_i*j_tot/F
        +a* (1+trans_i)*j_SEI/F
        -a*(j_SEI*V_bar_EC+j_tot*V_bar_Li-0.5*j_SEI*V_bar_SEI)*c_EC_init/F
    )
} 

In [ ]:
print( epislon**b *D_e )
print( epislon**b *D_cross*(c_init/c_tot_init))
print( a*(1-t_0plus)*j_tot/F
    - a*sign_2*(j_SEI*V_bar_EC+j_tot*V_bar_Li-0.5*j_SEI*V_bar_SEI)*c_init/F)

concatenation(broadcast(3.75e-11), broadcast(9.666472986565472e-11), broadcast(5.816858043652089e-11))
concatenation(broadcast(-3.1249999999999995e-11), broadcast(-8.055394155471226e-11), broadcast(-4.847381703043407e-11))
concatenation(broadcast(1.9136951216600957), broadcast(-0.0), broadcast(-2.3745495341343483))


In [ ]:
print( epislon**b *D_cross*(c_EC_init/c_tot_init) )
print( epislon**b *D_EC   )
print( sign_2 * (
        a*trans_i*j_tot/F
        +a* (1+trans_i)*j_SEI/F
        -a*(j_SEI*V_bar_EC+j_tot*V_bar_Li-0.5*j_SEI*V_bar_SEI)*c_EC_init/F
    ))

concatenation(broadcast(-1.2499999999999998e-10), broadcast(-3.2221576621884903e-10), broadcast(-1.9389526812173628e-10))
concatenation(broadcast(6.25e-11), broadcast(1.6110788310942454e-10), broadcast(9.694763406086815e-11))
concatenation(broadcast(-4.045348278871794), broadcast(0.0), broadcast(0.0))


In [ ]:
model.initial_conditions = {c_e: pb.Scalar(c_init),c_EC: pb.Scalar(c_EC_init)}

model.boundary_conditions = {
    c_e: {"left": (pb.Scalar(0), "Neumann"),"right": (pb.Scalar(0), "Neumann")},  
    c_EC:{"left": (pb.Scalar(0), "Neumann"),"right": (pb.Scalar(0), "Neumann")} }
# define geometry
x_n = pb.SpatialVariable(
    "x_n", 
    domain=["negative electrode"], 
    coord_sys="cartesian"
)
x_s = pb.SpatialVariable(
    "x_s", 
    domain=["separator"], 
    coord_sys="cartesian"
)
x_p = pb.SpatialVariable(
    "x_p", 
    domain=["positive electrode"], 
    coord_sys="cartesian"
)
geometry = {
    "negative electrode": {x_n: {"min": pb.Scalar(0), "max": pb.Scalar(8.52e-5)}},
    "separator":          {x_s: {"min": pb.Scalar(8.52e-5), "max": pb.Scalar(9.72e-5)}},
    "positive electrode": {x_p: {"min": pb.Scalar(9.72e-5), "max": pb.Scalar(17.28e-5)}},
}
# mesh and discretise ,"separator","positive electrode"
submesh_types = {
    "negative electrode": pb.MeshGenerator(pb.Uniform1DSubMesh),
    "separator": pb.MeshGenerator(pb.Uniform1DSubMesh),
    "positive electrode": pb.MeshGenerator(pb.Uniform1DSubMesh),
}
var_pts = {x_n: 20, x_s:20, x_p:20}
mesh = pb.Mesh(geometry, submesh_types, var_pts)
spatial_methods = {
    "negative electrode": pb.FiniteVolume(),
    "separator": pb.FiniteVolume(),
    "positive electrode": pb.FiniteVolume(),
}
disc = pb.Discretisation(mesh, spatial_methods)
disc.process_model(model);

In [ ]:
t = np.linspace(0, 10, 5)
print(t)

[ 0.   2.5  5.   7.5 10. ]


In [ ]:
# solve
solver = pb.ScipySolver()
#t = np.linspace(0, 100, 50)
solution = solver.solve(model, [0,1.2])


In [ ]:
pb.dynamic_plot(
    solution,
    [
        "Li+ concentration",
        "EC concentration",
    ],
)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.2, step=0.012), Output()), _dom_classes=('…